# Python program for Huffman Coding in image processing

## Prerequisites

Make sure you have the following libraries installed:
- `open-cv`
- `numpy`
- `tabulate`
- `heapq`

In [5]:
import cv2
import numpy as np
from collections import Counter, defaultdict
import heapq
from tabulate import tabulate

# Helper function to calculate probabilities
def calculate_probabilities(pixel_values):
    total_pixels = len(pixel_values)
    counter = Counter(pixel_values)
    probabilities = {k: v / total_pixels for k, v in counter.items()}
    return probabilities

# Huffman tree node
class Node:
    def __init__(self, freq, symbol, left=None, right=None):
        self.freq = freq
        self.symbol = symbol
        self.left = left
        self.right = right
        self.huff = ''

    def __lt__(self, nxt):
        return self.freq < nxt.freq

# Function to generate Huffman codes
def huffman_encoding(probabilities):
    heap = [Node(freq, symbol) for symbol, freq in probabilities.items()]
    heapq.heapify(heap)

    while len(heap) > 1:
        left = heapq.heappop(heap)
        right = heapq.heappop(heap)

        left.huff = '0'
        right.huff = '1'

        # Ensure the sum of frequencies does not overflow
        new_freq = left.freq + right.freq
        if new_freq < left.freq or new_freq < right.freq:
            raise OverflowError("Frequency overflow encountered")

        newNode = Node(new_freq, left.symbol + right.symbol, left, right)
        heapq.heappush(heap, newNode)

    huffman_dict = {}
    def generate_huffman_code(node, val=''):
        newVal = val + node.huff

        if node.left is None and node.right is None:
            huffman_dict[node.symbol] = newVal
            return

        generate_huffman_code(node.left, newVal)
        generate_huffman_code(node.right, newVal)

    root = heapq.heappop(heap)
    generate_huffman_code(root)
    return huffman_dict

# Function to print the results in a table
def print_huffman_table(probabilities, huffman_dict):
    table = []
    for symbol, prob in probabilities.items():
        table.append([symbol, prob, huffman_dict[symbol]])
    
    print(tabulate(table, headers=["Symbol", "Probability", "Huffman Code"], tablefmt="fancy_grid"))

# Load the image
image_path = '../images/input_image.jpeg'  # Change this to the path of your image
image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

# Flatten the image to a 1D array
pixels = image.flatten()

# Calculate probabilities
probabilities = calculate_probabilities(pixels)

# Apply Huffman encoding
huffman_dict = huffman_encoding(probabilities)

# Print the final result in a table
print_huffman_table(probabilities, huffman_dict)

╒══════════╤═══════════════╤═══════════════════════╕
│   Symbol │   Probability │          Huffman Code │
╞══════════╪═══════════════╪═══════════════════════╡
│      149 │   0.00678457  │               0110011 │
├──────────┼───────────────┼───────────────────────┤
│      152 │   0.0056982   │               0001000 │
├──────────┼───────────────┼───────────────────────┤
│      156 │   0.00628578  │               0100001 │
├──────────┼───────────────┼───────────────────────┤
│      158 │   0.00537791  │              11100100 │
├──────────┼───────────────┼───────────────────────┤
│      160 │   0.00479519  │              11000101 │
├──────────┼───────────────┼───────────────────────┤
│      163 │   0.00492728  │              11001110 │
├──────────┼───────────────┼───────────────────────┤
│      167 │   0.0056823   │               0000101 │
├──────────┼───────────────┼───────────────────────┤
│      170 │   0.00677131  │               0110010 │
├──────────┼───────────────┼──────────────────

C:\Users\gaura\AppData\Local\Temp\ipykernel_8628\3136827872.py:43: RuntimeWarning: overflow encountered in scalar add
  newNode = Node(new_freq, left.symbol + right.symbol, left, right)
